In [ ]:
# This code will attempt to subtract off a PSF
# From a galaxy with a known AGN
# 205 - 1 is a good example of this
# I am trying to make a version of it that takes up less space on the SC (fully convolved etc)


In [ ]:
# This cell is how I tried to do this before
'''find the spaxel of maximum brightness, this is the AGN light'''

                plt.clf()
                summed_specs = np.sum(out,axis=0)
		coords = np.argwhere(summed_specs.max() == summed_specs)
            #max_coord = np.where(summed_specs = np.max(summed_specs))                                                                
            #print('max coords', coords)                                                                                              
		plt.plot(Ang, out[:,coords[0][0],coords[0][1]], label='Simulated Spectrum', lw=0.5)

                plt.savefig('brightest_spaxel_'+str(run)+'_'+str(myr)+'_'+str(view)+'.png', dpi=1000)


In [ ]:
# My new idea is to use one spaxel, an also to use a stellar template if necessary.